In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
datasetPath = os.listdir('signatures')

signatureTypes = os.listdir('signatures')

print(datasetPath)

['full_forg', 'full_org']


In [3]:
signatures = []

for item in signatureTypes:
    allSignatures = os.listdir('signatures'+'/'+item)

    for signature in allSignatures:
        if signature.endswith(".png"): 
            signatures.append((item, str('signatures'+'/'+item)+'/'+signature))

print(signatures)


[('full_forg', 'signatures/full_forg/forgeries_10_1.png'), ('full_forg', 'signatures/full_forg/forgeries_10_10.png'), ('full_forg', 'signatures/full_forg/forgeries_10_11.png'), ('full_forg', 'signatures/full_forg/forgeries_10_12.png'), ('full_forg', 'signatures/full_forg/forgeries_10_13.png'), ('full_forg', 'signatures/full_forg/forgeries_10_14.png'), ('full_forg', 'signatures/full_forg/forgeries_10_15.png'), ('full_forg', 'signatures/full_forg/forgeries_10_16.png'), ('full_forg', 'signatures/full_forg/forgeries_10_17.png'), ('full_forg', 'signatures/full_forg/forgeries_10_18.png'), ('full_forg', 'signatures/full_forg/forgeries_10_19.png'), ('full_forg', 'signatures/full_forg/forgeries_10_2.png'), ('full_forg', 'signatures/full_forg/forgeries_10_20.png'), ('full_forg', 'signatures/full_forg/forgeries_10_21.png'), ('full_forg', 'signatures/full_forg/forgeries_10_22.png'), ('full_forg', 'signatures/full_forg/forgeries_10_23.png'), ('full_forg', 'signatures/full_forg/forgeries_10_24.png')

In [4]:
signaturesDF = pd.DataFrame(data=signatures,columns=['signature type','image'])
print(signaturesDF.head)

<bound method NDFrame.head of      signature type                                     image
0         full_forg   signatures/full_forg/forgeries_10_1.png
1         full_forg  signatures/full_forg/forgeries_10_10.png
2         full_forg  signatures/full_forg/forgeries_10_11.png
3         full_forg  signatures/full_forg/forgeries_10_12.png
4         full_forg  signatures/full_forg/forgeries_10_13.png
...             ...                                       ...
2635       full_org      signatures/full_org/original_9_5.png
2636       full_org      signatures/full_org/original_9_6.png
2637       full_org      signatures/full_org/original_9_7.png
2638       full_org      signatures/full_org/original_9_8.png
2639       full_org      signatures/full_org/original_9_9.png

[2640 rows x 2 columns]>


In [5]:
path = 'signatures/'
imSize = 227

images = []
labels = []

for i in signatureTypes:
    dataPath = path + str(i)
    filenames = [i for i in os.listdir(dataPath)]

    for f in filenames:
        if f.endswith(".png"): 
            img = cv2.imread(dataPath+'/'+f)
            img = cv2.resize(img,(imSize,imSize))
            images.append(img)
            labels.append(i)

labels
images

[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [253, 253, 253],
         [254, 254, 254],
         [230, 230, 230]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [252, 252, 252],
         [253, 253, 253],
         [234, 234, 234]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [254, 254, 254],
         [255, 255, 255],
         [238, 238, 238]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [254, 254, 254],
         [253, 253, 253]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [253, 253, 253],
         [253, 253, 253]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [6]:
images = np.array(images)

images.shape

(2640, 227, 227, 3)

In [7]:
images = images.astype('float32')/255.0

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

y = signaturesDF['signature type'].values

y = y.reshape(-1,1)

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
Y=np.array(columnTransformer.fit_transform(y),dtype=np.str)

Y

C:\Users\ASUS\AppData\Local\Temp\ipykernel_9056\3871765133.py:9: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y=np.array(columnTransformer.fit_transform(y),dtype=np.str)


array([['1.0', '0.0'],
       ['1.0', '0.0'],
       ['1.0', '0.0'],
       ...,
       ['0.0', '1.0'],
       ['0.0', '1.0'],
       ['0.0', '1.0']], dtype='<U32')

In [9]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, Y = shuffle(images,Y,random_state=1)

trainX,testX,trainY,testY = train_test_split(images, Y, test_size=0.2, random_state=415)

print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(2112, 227, 227, 3)
(528, 227, 227, 3)
(2112, 2)
(528, 2)


In [22]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
numClasses = 2

x = tf.placeholder(tf.float32,shape=[None,227,227,3])

y_ = tf.placeholder(tf.float32,numClasses)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [11]:

num_classes=2
weights = {
    'w1': tf.Variable(tf.random_normal([11, 11, 3, 96]),name='w1'),  # 11*11*3   96 filters
    'w2': tf.Variable(tf.random_normal([5, 5, 96, 256]),name='w2'), # 5*5*96  256 filters 
    
    'w3': tf.Variable(tf.random_normal([3, 3, 256, 384]),name='w3'), 
    'w4': tf.Variable(tf.random_normal([3, 3, 384, 384]),name='w4'),
    
    'w5': tf.Variable(tf.random_normal([3, 3, 384, 256]),name='w5'),  
    
    
    'wfc1': tf.Variable(tf.random_normal([6 * 6 * 256, 4096]),name='wfc1'),
    'wfc2': tf.Variable(tf.random_normal([4096, 4096]),name='wfc2'), 
    
    'wout': tf.Variable(tf.random_normal([4096, 2]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random_normal([96]),name='b1'),
    'b2': tf.Variable(tf.random_normal([256]),name='b2'),
    
    'b3': tf.Variable(tf.random_normal([384]),name='b3'),
    'b4': tf.Variable(tf.random_normal([384]),name='b4'),
    
    'b5': tf.Variable(tf.random_normal([256]),name='b5'),
    
    'bfc1': tf.Variable(tf.random_normal([4096]),name='bfc1'),  #4096
    'bfc2': tf.Variable(tf.random_normal([4096]),name='bfc2'),  #4096
    'bout': tf.Variable(tf.random_normal([num_classes]),name='bout') # 2
}

In [12]:
def alex_net(x, weights, biases):
  # reshape input to 227*227*3 size
 x = tf.reshape(x, shape=[-1, 227, 227, 3])  
    
 print("###########################################################################")
 print("size of x is")
 print(x.shape)
    


 #conv1
 #kernel size=11*11, stride=4
 
 conv1_in = tf.nn.conv2d(x, weights['w1'], strides=[1, 4, 4, 1], padding="SAME")
 conv1_in = tf.nn.bias_add(conv1_in,  biases['b1'])   # y= xw+b
 conv1 = tf.nn.relu(conv1_in)
 #maxpool1   #kernel size 3*3, stride 2

 maxpool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
 print("###########################################################################")
 print("size after 1st conv layer) is ")
 print(maxpool1.shape)
    

 #conv2
 # kernel 5*5, channels 256, stride 1
 conv2_in = tf.nn.conv2d(maxpool1,weights['w2'], strides=[1, 1, 1, 1], padding="SAME")
 conv2_in = tf.nn.bias_add(conv2_in,  biases['b2'])   
 conv2 = tf.nn.relu(conv2_in)

 #maxpool2  
 #kernel 3*3 stride 2
 maxpool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")
 print("###########################################################################")
 print("size after 2nd conv layer) is ")
 print(maxpool2.shape)
  
 #conv3
 #kernel size=3*3, channels=384, stride 1   
 conv3_in = tf.nn.conv2d(maxpool2, weights['w3'],strides=[1, 1, 1, 1], padding="SAME")
 conv3_in = tf.nn.bias_add(conv3_in,  biases['b3'])  
 conv3 = tf.nn.relu(conv3_in)
 print("###########################################################################")
 print("size after 3rd conv layer) is ")
 print(conv3.shape)
     

 #conv4
 # kernel=3*3,channels=384,stride=1   
 conv4_in = tf.nn.conv2d(conv3, weights['w4'], strides=[1, 1, 1, 1], padding="SAME")
 conv4_in = tf.nn.bias_add(conv4_in,  biases['b4'])  
 conv4 = tf.nn.relu(conv4_in)
 print("###########################################################################")
 print("size after 4th conv layer) is ")
 print(conv4.shape)

 #conv5
 # kernel 3*3, channels=256, stride 1   
 conv5_in = tf.nn.conv2d(conv4, weights['w5'],strides=[1, 1, 1, 1], padding="SAME")
 conv5_in = tf.nn.bias_add(conv5_in,  biases['b5']) 
 conv5 = tf.nn.relu(conv5_in)
 
 #maxpool5  # kernel 3*3, stride 2
 maxpool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="VALID")

 print("###########################################################################")
 print("size after 5th conv layer) is ")
 print(conv5.shape)
 print("###########################################################################")
 print("size after 5th conv layer pooling) is ")
 print(maxpool5.shape)


 fc6 = tf.reshape(maxpool5, [-1, weights['wfc1'].get_shape().as_list()[0]])
 print("###########################################################################")
 print("size after reshaping the image ) is ")
 print(fc6.shape) 

 fc6 = tf.add(tf.matmul(fc6, weights['wfc1']), biases['bfc1'])
 fc6 = tf.nn.relu(fc6)
      

 #fc7
 fc7 = tf.nn.relu_layer(fc6, weights['wfc2'], biases['bfc2'])
 print("###########################################################################")
 print("size after 7th layer) is ")
 print(fc7.shape)


 fc8 = tf.add(tf.matmul(fc7, weights['wout']), biases['bout'])
 out = tf.nn.softmax(fc8)
    
 return out   

 

In [13]:
# Create the model
model = alex_net(x, weights, biases)
print(model)

###########################################################################
size of x is
(?, 227, 227, 3)
###########################################################################
size after 1st conv layer) is 
(?, 28, 28, 96)
###########################################################################
size after 2nd conv layer) is 
(?, 13, 13, 256)
###########################################################################
size after 3rd conv layer) is 
(?, 13, 13, 384)
###########################################################################
size after 4th conv layer) is 
(?, 13, 13, 384)
###########################################################################
size after 5th conv layer) is 
(?, 13, 13, 256)
###########################################################################
size after 5th conv layer pooling) is 
(?, 6, 6, 256)
###########################################################################
size after reshaping the image ) is 
(?, 9216)
######################

In [14]:
learning_rate=0.01
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
init = tf.initialize_all_variables()


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [19]:
cost_history=[]
n_epochs =5
# the execution
sess = tf.Session()
sess.run(init)

trainY=trainY.toarray()


for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: trainX, y_: trainY})  #working
    cost_history = np.append(cost_history,c)  # working
    print('epoch : ', i,  ' - ', 'cost: ', c) #working 


AttributeError: 'numpy.ndarray' object has no attribute 'toarray'